### OpenAI Assistant Agent

This notebook shows how to consume [`OpenAI Assistant API`](https://platform.openai.com/docs/assistants/overview) using `OpenAIAssistantAgent` class.

In [1]:
// Install dependencies
#i "nuget:https://www.myget.org/F/agentchat/api/v3/index.json"

#r "nuget:AgentChat.OpenAI,*-*"

Restore sources https://www.myget.org/F/agentchat/api/v3/index.json Installed Packages AgentChat.OpenAI, 0.4.2

#### Create `OpenAIAssistantAgent`

You can either create a new `OpenAIAssistantAgent`, which create a new `OpenAI Assistant` on the server, or load an existing `OpenAI Assistant`.

In [2]:
using AgentChat.OpenAI;
using Azure.AI.OpenAI;

var apiKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY") ?? throw new Exception("OPENAI_API_KEY not set");
var openAIClient = new OpenAIClient(apiKey);
var assistant = await OpenAIAssistantAgent.CreateAsync(
    openAIClient,
    name: "assistant",
    roleInformation: "You are a helpful AI assistant",
    description: "A helpful AI assistant that will answer your questions",
    model: "gpt-3.5-turbo",
    useCodeInterpreter: false,
    useRetrieval: false);

// retrieve existing assistant
var id = assistant.ID;
var assistantObject = await openAIClient.RetrieveAssistantAsync(id);
var retrievedAssistant = new OpenAIAssistantAgent(openAIClient, assistantObject);

#### Interact with `OpenAIAssistantAgent`

In [3]:
using AgentChat;
var reply = await assistant.SendMessageAsync("Hello, how are you?");
reply.PrettyPrintMessage();

Message from assistant
--------------------
Hello! I'm an AI assistant, so I don't have feelings in the same way humans do. But I'm here and ready to assist you with anything you need! How can I help you today?

--------------------



#### use code interpreter to resolve task via coding

You can enable `OpenAI code_interpreter` tool in `OpenAIAssistantAgent` by setting `useCodeInterpreter` to true

In [4]:
var question = "what's 33th of fibonacci minus 23th of fibonacci"; //  3,495,921
// agent without code interpreter
var reply = await assistant.SendMessageAsync(question);
reply.PrettyPrintMessage();

// agent with code interpreter
var assistantWithCodeInterpreter = await OpenAIAssistantAgent.CreateAsync(
    openAIClient,
    name: "assistant",
    roleInformation: "You are a helpful AI assistant",
    model: "gpt-3.5-turbo",
    description: "A helpful AI assistant that will answer your questions",
    useCodeInterpreter: true,
    useRetrieval: false);
reply = await assistantWithCodeInterpreter.SendMessageAsync(question);
reply.PrettyPrintMessage();

Message from assistant
--------------------
To calculate the 33rd Fibonacci number, we can use the formula Fn = Fn-1 + Fn-2, where F1 = 1 and F2 = 1.

Using this formula, we can find that the 33rd Fibonacci number is 3524578.

Similarly, the 23rd Fibonacci number is 28657.

To find the difference between the 33rd and 23rd Fibonacci numbers:

3524578 - 28657 = 3495921

Therefore, the 33rd Fibonacci minus the 23rd Fibonacci is 3,495,921.

--------------------

Message from assistant
--------------------
To find the 33rd Fibonacci number, we need to calculate the sum of the 32nd and 31st Fibonacci numbers. Similarly, to find the 23rd Fibonacci number, we need to calculate the sum of the 22nd and 21st Fibonacci numbers. We can subtract the 23rd Fibonacci number from the 33rd Fibonacci number to get the result.
The result of the 33rd Fibonacci number minus the 23rd Fibonacci number is 3,495,921.

--------------------



#### Custom function call
You can enable Function calling in `OpenAIAssistantAgent` by passing a `FunctionMap` when creating `OpenAIAssistantAgent`

In [5]:
using System.Text.Json;

var guessNumberFunctionDefinition = new FunctionDefinition
	{
		Name = @"GuessNumber",
        Description = """
Guess integer number between 1 and 10
""",
        Parameters = BinaryData.FromObjectAsJson(new
        {
            Type = "object",
            Properties = new
			{
			    number = new
			    {
				    Type = @"number",
				    Description = @"number to guess",
			    },
            },
            Required = new []
			{
			    "number",
			},
        },
        new JsonSerializerOptions
		{
			PropertyNamingPolicy = JsonNamingPolicy.CamelCase,
		})
    };

var guessNumberFunc = async (string args) => {
	var obj = JsonSerializer.Deserialize<JsonElement>(args);
	var number = obj.GetProperty("number").GetInt32();
	Console.WriteLine($"number: {number}");
	if (number < 1 || number > 10)
	{
		Console.WriteLine("Number must be between 1 and 10");
		return "Number must be between 1 and 10";
	}

	if (number == 7)
	{
		Console.WriteLine("You guessed it!");
		return "You guessed it!";
	}

	if (number < 7)
	{
		Console.WriteLine("Too low");
		return "Too low";
	}

	Console.WriteLine("Too high");
	return "Too high";
};

var assistant = await OpenAIAssistantAgent.CreateAsync(
	client: openAIClient,
	name: "assistant",
	roleInformation: "You are a helpful AI assistant",
	model: "gpt-3.5-turbo",
	description: "A helpful AI assistant that will answer your questions",
	useCodeInterpreter: false,
	useRetrieval: false,
	functionMaps: new Dictionary<FunctionDefinition, Func<string, Task<string>>>
	{
		{ guessNumberFunctionDefinition, guessNumberFunc },
	}
);

var reply = await assistant.SendMessageAsync("guess a integer number between 1 and 10, and tell me the number once you guess it");

reply.PrettyPrintMessage();


number: 5
Too low
number: 8
Too high
number: 6
Too low
number: 7
You guessed it!
Message from assistant
--------------------
The number is 7.

--------------------



#### Remove `OpenAIAssistantAgent`

You can remove the `OpenAI Assistant` from portal by calling `RemoveAssistantAsync`

In [6]:
// remove assistant
await openAIClient.RemoveAssistantAsync(assistant.ID);

// remove assistant with code interpreter
await openAIClient.RemoveAssistantAsync(assistantWithCodeInterpreter.ID);